# Импорты

In [48]:
import pandas as pd
import numpy as np
import cv2
from os import listdir
from os.path import isfile, join
import torch
from sklearn.cluster import KMeans
from collections import Counter

# Task #1 - 10 баллов

In [1]:
pip install cv2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement cv2 (from versions: none)
ERROR: No matching distribution found for cv2


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
def merge_channels(input_dir, output_dir):
  df = pd.read_csv(f'{input_dir}/description.csv') # считываем метадату
  f = open(f'{input_dir}/image_counter.txt', 'r')

  n = int(f.read())
  f.close()

  for i in range(1,n+1):          
    names = df[df['full_image_index']==i]['image_path'].values          #выделяем дату для изображения
    image_b   = cv2.imread(f'{input_dir}/data/{names[0]}')  # считываем изображения
    image_g   = cv2.imread(f'{input_dir}/data/{names[1]}')
    image_r   = cv2.imread(f'{input_dir}/data/{names[2]}')

    image_out = image_b
    for о in range(len(image_out)):       # соединяем (cv2.merge не работал)
      for j in range(len(image_out[о])):
        image_out[о][j][1] = image_g[о][j][1]
        image_out[о][j][2] = image_r[о][j][2]
    cv2.imwrite(f'{output_dir}/{i}_image.jpg',image_out)   # записываем

In [5]:
merge_channels('/content/drive/MyDrive/python_split_image_by_ch','/content/drive/MyDrive/output_cv_1')

# Task №2 - 20 баллов

In [43]:
def find_car(input_dir, output_cars ="output.csv"):
  df = pd.DataFrame(columns=['name_file','flag_car'])
  model = torch.hub.load('ultralytics/yolov5', 'yolov5s')
  only_files = [f for f in listdir(input_dir) if isfile(join(input_dir, f))]
  df = pd.DataFrame(columns=['name_file','flag_car'])

  for i in only_files:
    detected = model(f'{input_dir}/{i}').pandas().xyxy[0]['name'].values
    flag =('bus' in detected) or ('car' in detected) or ('truck' in detected)
    row = {'name_file': i,'flag_car': flag}
    df = df.append(row,ignore_index=True)
  df.to_csv(output_cars)
  

In [44]:
find_car('/content/drive/MyDrive/output_cv_1')

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-6-18 Python-3.7.13 torch-1.11.0+cu113 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [45]:
df_flag = pd.read_csv('/content/drive/MyDrive/python_split_image_by_ch/val.csv')
df_predict = pd.read_csv('/content/output.csv')
df_flag.columns = ['name_file','flag_car']

In [46]:
cnt = 0
for i in range(df_flag.shape[0]):
  if df_flag.iloc[i]['flag_car'] == df_predict.iloc[i]['flag_car']:
    cnt +=1
print(cnt,' image equal of', df_flag.shape[0])

53  image equal of 99


# Task №3 - 30 баллов

In [49]:
pip install sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [60]:
def calc_metric(image, x, y, w, h):
    crop_img = image[y:y+h, x:x+w]
    crop_img = crop_img.reshape((crop_img.shape[0] * crop_img.shape[1], 3))
    clt = KMeans(n_clusters = 3)
    labels = clt.fit_predict(crop_img)
    label_counts = Counter(labels)
    dominant_color = clt.cluster_centers_[label_counts.most_common(1)[0][0]]
    return list(dominant_color)

In [62]:
img = cv2.imread('/content/drive/MyDrive/output_cv_1/5_image.jpg')
dom_color = calc_metric(img,0,0,500,500)
print(dom_color)
dom_color_hsv = np.full(img.shape, dom_color, dtype='uint8')
dom_color_bgr = cv2.cvtColor(dom_color_hsv, cv2.COLOR_HSV2BGR)
output_image = np.hstack((img, dom_color_bgr))
cv2.imwrite('/content/dom_color.jpg',output_image)

[251.72950438501272, 232.5639423732015, 159.8852489818518]


True

# Task 4 - 40 баллов


In [115]:
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from collections import Counter
colors = {
    'red':(237,28,38),
    'yellow':(255,242,0),
    'white_silver':(127,127,127),
    'green':(34,177,76),
    'blue_cyan':(203,228,253),
    'black':(0,0,0)
}
def category_color(color_buf):
  colors_diff = {
    'red':0,
    'yellow':0,
    'white_silver':0,
    'green':0,
    'blue_cyan':0,
    'black':0
  }  
  for color in colors:
    colors_diff[color] = np.sqrt( (colors[color][0]-color_buf[0])**2 + (colors[color][1]-color_buf[1])**2 + (colors[color][2]-color_buf[2])**2 )
  return min(colors_diff, key=colors_diff.get)

def find_color(input_dir, output_file='output_color.csv'):
  df = pd.read_csv(f'{input_dir}/description.csv') # считываем метадату
  f = open(f'{input_dir}/image_counter.txt', 'r')
  n = int(f.read())
  f.close()
  colors = []
  model = torch.hub.load('ultralytics/yolov5', 'yolov5s')   # находим машину
  col_names= []
  for i in range(1,n+1):          
    names = df[df['full_image_index']==i]['image_path'].values          #выделяем дату для изображения
    
    image_b   = cv2.imread(f'{input_dir}/data/{names[0]}')  # считываем изображения
    image_g   = cv2.imread(f'{input_dir}/data/{names[1]}')
    image_r   = cv2.imread(f'{input_dir}/data/{names[2]}')
    col_names.append(f'{names[0][:5]}.jpg')
    image_out = image_b
    for о in range(len(image_out)):       # соединяем (cv2.merge не работал)
      for j in range(len(image_out[о])):
        image_out[о][j][1] = image_g[о][j][1]
        image_out[о][j][2] = image_r[о][j][2]
   
    detected = model(image_out).pandas().xyxy[0]
    list_names = ['car','bus','truck']
    for j in list_names:
     if j in detected['name'].unique():
        x_min = int(detected[detected['name']==j]['xmin'].values[0])      # сохраняем координаты машины
        x_max = int(detected[detected['name']==j]['xmax'].values[0])
        y_min = int(detected[detected['name']==j]['ymin'].values[0])
        y_max = int(detected[detected['name']==j]['ymax'].values[0])
        break
    crop_img = image_out[y_min:y_max, x_min:x_max]              # обрезаем
    crop_img = crop_img.reshape((crop_img.shape[0] * crop_img.shape[1], 3))
    clt = KMeans(n_clusters = 3)
    labels = clt.fit_predict(crop_img)
    label_counts = Counter(labels)
    dominant_color = clt.cluster_centers_[label_counts.most_common(1)[0][0]]
    colors.append(category_color(list(dominant_color)))
  df_output = pd.DataFrame(columns = ['file_name','color'])
  df_output['file_name'] = col_names
  df_output['color'] = colors
  df_output.to_csv(output_file)

In [116]:
find_color('/content/drive/MyDrive/color_cars_train')

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-6-18 Python-3.7.13 torch-1.11.0+cu113 CPU

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 


In [117]:
df_predict_colors = pd.read_csv('output_color.csv')
df_predict_colors

,Unnamed: 0,file_name,color
0,0,00001.jpg,green
1,1,00002.jpg,white_silver
2,2,00003.jpg,blue_cyan
3,3,00004.jpg,black
4,4,00005.jpg,black
...,...,...,...
91,91,00092.jpg,black
92,92,00093.jpg,white_silver
93,93,00094.jpg,black
94,94,00095.jpg,black


In [118]:
df_fact_colors = pd.read_csv('/content/drive/MyDrive/color_cars_train/colors.csv')
df_fact_colors.columns=['file_name','color']
df_fact_colors

,file_name,color
0,00002.jpg,yellow
1,00003.jpg,green
2,00004.jpg,black
3,00005.jpg,white_silver
4,00006.jpg,red
...,...,...
90,00092.jpg,blue_cyan
91,00093.jpg,blue_cyan
92,00094.jpg,white_silver
93,00095.jpg,white_silver


In [119]:
cnt = 0
for i in range(df_fact_colors.shape[0]):
  if df_fact_colors.iloc[i]['color'] == df_predict_colors.iloc[i]['color']:
    cnt +=1
print(cnt,' images with equal color of', df_flag.shape[0])

13  images with equal color of 99
